In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import natsort
import glob
import itertools
import numpy as np
import pandas as pd
from torchvision import models
from PIL import Image
import matplotlib.pyplot as plt
import timm
from sklearn.model_selection import train_test_split
from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import KFold,StratifiedKFold


import torch.optim as optim
from torch.utils.data import Dataset,DataLoader

import torchvision
import torchvision.transforms as transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1218)
if device =='cuda':
    torch.cuda.manual_seed_all(1218)
print(device)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1218)
if device =='cuda':
    torch.cuda.manual_seed_all(1218)
print(device)

In [ ]:
model_incep3=models.inception_v3(aux_logits=False,pretrained=True)
incep_num_ftrs = model_incep3.fc.in_features
model_incep3.fc = nn.Linear(incep_num_ftrs, num_class)
model_incep3.to(device)
print(model_incep3)

In [ ]:
model_effi=EfficientNet.from_pretrained('efficientnet-b4')
num_class = 7
effi_num_ftrs = model_effi._fc.in_features
model_effi._fc = nn.Linear(effi_num_ftrs, num_class)
model_effi.to(device)

In [ ]:
trans = transforms.Compose([
#     transforms.RandomRotation(30),
    transforms.CenterCrop(224),
#     transforms.Grayscale(num_output_channels=3),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.Resize((224)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = torchvision.datasets.ImageFolder('your_path',transform=trans)

In [ ]:
train_data,val_data=train_test_split(dataset, test_size=0.1)

In [ ]:
train_loader = DataLoader(dataset=train_data,batch_size=32,shuffle=True)
val_loader = DataLoader(dataset=val_data,batch_size=32,shuffle=True)

In [ ]:
loss_func=torch.nn.CrossEntropyLoss()

In [ ]:
epochs = 5
model=0
model_lst=['res50','incep3','effi']
n_sample = 10


for model_idx in range(3):
    if(model_idx==0):
        model=model_res50
    elif (model_idx==1):
        model=model_incep3
    elif (model_idx==2):
        model=model_effi
    
    optimizer = optim.Adam(model.parameters(),lr=0.00002)
    lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.0001)
    
    for epoch in range(epochs):  # loop over the dataset multiple times
        print("------- train -------")
        running_loss = 0.0
        train_total = 0
        train_correct=0
        lr_sche.step()
        for i, data in enumerate(train_loader):
            # get the inputs
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = loss_func(outputs, labels)
            predict = torch.argmax(outputs, dim=1)
            loss.backward()
            optimizer.step() 
        
            # print statistics
            running_loss += loss.item()
            train_total += labels.size(0)
            train_correct += (predict==labels).sum()
            
            if i % 32 == 31:    
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 32))
            
                running_loss = 0.0
        print("[epoch: {}] Accuracy : {:.2f}".format(epoch+1,100*train_correct/train_total))
        print()
    
        with torch.no_grad():
            model.eval() # evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시키도록 하는 함수
            total=0
            correct=0

            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)
     
                out = model(inputs)
                predict = torch.argmax(out, dim=1)
            
        
                total += labels.size(0)
                correct += (predict==labels).sum()


            
        torch.save(model.state_dict(),'model{}:{}.pth'.format(model_idx,model_lst[model_idx]))    
        print("-- validation accuracy --")
        print("Accuracy : {:.2f}".format(100*correct/total))
        print()

print('Finish')

In [ ]:
class CustomDataSet(Dataset):

    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform

        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsort.natsorted(all_imgs)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        # img_loc = image path 정보 /shape/box/0/0.jpg
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        # image -> tensor

        return tensor_image

In [ ]:
test_data = CustomDataSet('your_path', transform=trans)

In [ ]:
test_set = DataLoader(dataset = test_data, batch_size = 32)

In [ ]:
save_lst=[]
model_res50.load_state_dict(torch.load('model0:res50.pth'))
model_incep3.load_state_dict(torch.load('model1:incep3.pth'))
model_effi.load_state_dict(torch.load('model2:effi.pth'))

save_lst.append(model_res50)
save_lst.append(model_incep3)
save_lst.append(model_effi)


In [ ]:
result =[]

with torch.no_grad():
    file=pd.read_csv('your_path.csv')
    
    for data in test_set:
        imgs = data
        imgs = imgs.to(device)
        pre_sum=0
        for i in range(3):
            predict=save_lst[i]
            prediction=predict(imgs)
            pre_sum=pre_sum+prediction
        
        result.append(torch.argmax(pre_sum,1).tolist())
        

    result_ = list(itertools.chain.from_iterable(result))
    prediction_np = np.array(result_)
    print(result_)

    file['value'] =result_
    file.to_csv('result.csv', index=False)       
